# Tutorial on Decorators 
--- 

## Why this topic ?

I consider this tutorial to be useful for at least three reasons:
1. Decorators are actively **used by third-party libraries**, like in the web-development frameworks Django, Flask e.t.c..
2. The very core idea of a decorator, which is primirily to **extend the functionality of an implementation**, can be used in your projects.
3. In order to understand how decorators work, you will need to **get a feeling on what is happening behind the scenes in Python**.

In my opinion, **the last point is the most crucial one**. There is such an extensive growth of open-source libraries and projects that we can   
use, that oftentimes you(we) really forget to appreciate **how** these libraries actually work. In the long run this shows up. 

> You will not be a able to debug, maintain or even extend the usage of your environment.   
> I read somewhere the phrase "**you are actually enslaved within these libraries**". 

Loosely speaking, **I categorize Python development in two branches: the pure Pythonic and the reflection-based implementations**. The latter   
incorporate usually a statically typed language, like Fortran or C/C++, in which your implementation is written and later exposed   
to be called from the Python interpreter. There, the benefits can be enormous with type-protection and speed being usually ensured, albeit     
your work becomes much harder despite the existence of many tools.  

Coming back in the pure Pythonic branch, one should find the **equilibrium between simplicity and effectiveness**. To do so you need to understand   
many diverse concepts ranging from plain-vanilla classes and data structures to design-patterns, architecture and testing. And this understanding   
relies, in the end, on realizing that **everything that you see and interact in Python can be manipulated almost at will**. The latter allows you to   
create complex but user friendly and maintainable applications, oftentimes as __effecient as necessary__, boost you workflow and   
eventually stop being a **slave**. 

**Disclaimer**: Although I write (often) in the first person, I hope that you don't get intimidated. There are too many concepts that   
I have no clue about and many more that I struggle to fully understand. 

> **And it is your job to speak, at some point,  in the first person and guide me through... **



## Overview of this tutorial

- **Motivation**: You already work with decorators
- **Section 1**: Decorators as functions  
- **Section 2**: Decorators as classes 
    * Decorators without arguments 
    * Decorators with arguments 
- **Section 3**: Creation of interface
    * Why you need to consider an interface
    * Example implementation to employ in production code
- **Section 4**: Implementation of a cache (+questions)
 

---


In [ ]:
import functools  # A very useful library!!
import time 
from datetime import datetime

## Motivation: You already work with decorators

Already in one of the simplest possible classes decorators might come into play  
Here we use the @property and @classmethod decorators  
- `@property`: 'squared' can be called now as if it was an attribute of the instance (but it is not!!)  
- `@classmethod`: the routines that are decorated as such, are often used as a 'factory' pattern   



In [ ]:
class SimpleNumber(object):
    """
    A class to represent a float or an integer number
    
    Arguments
    ---------
    x: integer or float
    
    """
    def __init__(self, number):
        if not isinstance(number, (int, float)):
            raise TypeError("Wrong type given. x: {}".format(type(number)))
        self.number = number
    
    @property
    def squared(self):
        """Gives you the squared of x"""
        return self.number**2
    
    @staticmethod
    def multiply_by_two(value):
        """Example of a staticmethod: Does not use instance attributes"""
        return 2*value 
    
    @classmethod
    def from_string(cls, number_as_string, astype=float):
        """Construct an instance by giving a number as a string"""
        if not isinstance(number_as_string, str):
            raise TypeError("Wrong type given. x: {}".format(type(number_as_string)))
        else:
            print("- Casting string-input to {}".format(astype))
        return cls(astype(number_as_string))
    
    def __str__(self):
        """Called by the 'print' function"""
        return "An instance of SimpleNumber with x = {}".format(self.number)
    
    def __repr__(self):
        """Representation of the instance"""
        return self.__str__()
    
# Example cases 
S1 = SimpleNumber(10)           
S2 = SimpleNumber.from_string("10", float)
S3 = SimpleNumber.from_string("10", int)

### Remark
At this point you might argue that this is an overkill. In fact you would say that you could include the "string-based" creation already   
in the `__init__` method or overload the `__new__` one and just do the job. However, my advice is always to simplify any function call.   
From the user's perpective it is easier to understand **many** simple functions than **one** complicated one, as long as the documentation   
is properly posted. Additionally, not only you delegate part of the responsibility to the user but also you make your code much more extensible.

Regarding the `@property`: This is crucial most of the times in terms of memory-management. For instance, assume that `x`   
is a large matrix and you are going to need it's square only a few times at runtime. **DO NOT STORE IT AS ATTRIBUTE**.  
Instead **mask** it as if it is an attribute. Of course, this is only the tip of the iceberg regarding properties and one could   
create a small tutorial for them.

---

## Section 1: Decorators as functions  

Previously, I have shown you that you do actually use decorators already. Here, I will describe the pattern 
that decorators invoke in their usual functional form.

In principle, a `decorator` is just a function that takes an other function as an argument in order to extend   
it's usage. If you want to be precise, a function is a callback: an object (class) that implements the `__call__`    
build-in method. (we will discuss about it in the next section)

In [ ]:
# A Decorator, in principle, is nothing but a function that takes a 
# function as an argument and returns a function
def the_simplest_decorator(callback):
    """A simple decorator. Note that *args and **kwargs refer to the callback!!"""
    def _wrapped_callback(*args, **kwargs):
        print("- [Decorator]: Arguments parsed : {}".format(args))
        print("- [Decorator]: Keyword arguments: {}".format(kwargs))
        return callback(*args, **kwargs)
    return _wrapped_callback

That's it... This is a decorator that you can use as ANY other function

In [ ]:
# A test function 
def add_two_numbers(x, y):
    """Adds two numbers"""
    return x + y 

# Let's dress this function with our decorator
add_two_numbers_decorated = the_simplest_decorator(add_two_numbers)

# And let's see what is this object 
print("Object-type: ", type(add_two_numbers_decorated))

In [ ]:
# So we can simply use it as a function 
print("Our result 1 + 2 = {}".format(add_two_numbers_decorated(1,2)))

### Everything is very clear now: without changing the function `add_two_numbers` we have managed to extend it!! 

But still, all this renaming from `add_two_numbers` to `add_two_numbers_decorated` is ugly and error prone. 
Instead let's just simply reassign it 

In [ ]:
# A test function 
def add_two_numbers(x, y):
    """Adds two numbers"""
    return x + y 

# Let's call the decorator on it 
add_two_numbers = the_simplest_decorator(add_two_numbers)

# And let's see what is this object 
print("Object-type: ", type(add_two_numbers))

# So we can simply use it as a function 
print("Our result 1 + 2 = {}".format(add_two_numbers(1,2)))

### Much cleaner... But we can do it a bit better. This is where the `@` symbol comes into play (syntactic sugar)

In [ ]:
@the_simplest_decorator
def add_two_numbers(x, y):
    """Adds two numbers"""
    return x + y

# And once again we just call it
print("Our result 1 + 2 = {}".format(add_two_numbers(1,2))) 

### In fact we can even nest decorators as much as we want!!

In [ ]:
def the_simplest_decorator_1(callback):
    """A simple decorator. Note that *args and **kwargs refer to the callback!!"""
    def _wrapped_callback_1(*args, **kwargs):
        print("- [Decorator-1]: Arguments parsed : {}".format(args))
        print("- [Decorator-1]: Keyword arguments: {}".format(kwargs))
        return callback(*args, **kwargs)
    return _wrapped_callback_1

def the_simplest_decorator_2(callback):
    """A simple decorator. Note that *args and **kwargs refer to the callback!!"""
    def _wrapped_callback_2(*args, **kwargs):
        print("- [Decorator-2]: Arguments parsed : {}".format(args))
        print("- [Decorator-2]: Keyword arguments: {}".format(kwargs))
        return callback(*args, **kwargs)
    return _wrapped_callback_2

# Nested decorators
@the_simplest_decorator_1
@the_simplest_decorator_2
def add_two_numbers(x, y):
    """Adds two numbers"""
    return x + y

# And once again we just call it
print("Our result 1 + 2 = {}".format(add_two_numbers(1,2))) 

### Remarks

Everything seems fine, but in fact there are some 'hidden' (solvable) issues here... The 'internal' structure 
(data-model attributes) of our function are changed. This is something that you might not want 

In [ ]:
def add_two_numbers_1(x, y):
    """Adds two numbers (no decorator)"""
    return x + y

@the_simplest_decorator_1
def add_two_numbers_2(x, y):
    """Adds two numbers (with decorator)"""
    return x + y

@the_simplest_decorator_2
@the_simplest_decorator_1
def add_two_numbers_3(x, y):
    """Adds two numbers (with 2 decorators)"""
    return x + y

print("Not decorated function: ")
print("- docstring: %s"%add_two_numbers_1.__doc__)
print("- name     : %s"%add_two_numbers_1.__name__)
print("Decorated (1) function: ")
print("- docstring: %s"%add_two_numbers_2.__doc__)
print("- name     : %s"%add_two_numbers_2.__name__)
print("Decorated (2) function: ")
print("- docstring: %s"%add_two_numbers_3.__doc__)
print("- name     : %s"%add_two_numbers_3.__name__)

### This can be a serious problem. When decorating a function you lose valuable information that you might need, for example in logging. 

On the bright side however, this problem is solved by the standard library, that provides what else... a decorator to do the job!

In [ ]:
# This is the structure that your decorators should have in principle
def the_simplest_decorator_3(callback):
    """A simple decorator. Note that *args and **kwargs refer to the callback!!"""
    @functools.wraps(callback)
    def _wrapped_callback_3(*args, **kwargs):
        print("- [Decorator]: Arguments parsed : {}".format(args))
        print("- [Decorator]: Keyword arguments: {}".format(kwargs))
        return callback(*args, **kwargs)
    return _wrapped_callback_3


@the_simplest_decorator_3
def add_two_numbers_4(x, y):
    """Adds two numbers (with decorator and functools.wraps)"""
    return x + y

print("Decorated function: ")
print("- docstring    : %s"%add_two_numbers_4.__doc__)
print("- name         : %s\n"%add_two_numbers_4.__name__)

# And once again we just call it
print("Our result 1 + 2 = {}\n\n".format(add_two_numbers_4(1,2))) 

### Short summary

The decorator is nothing but a function that takes the form shown below:

```python
def decorator_name(callback):
    """Decorator docstring"""
    @functools.wraps(callback)
    def _wrapped_callback(*args, **kwargs):
        #            ........            
        # ... your implentation goes here ...
        #            ........            
        return callback(*args, **kwargs)
    return _wrapped_callback
```

To decorator a function
```python 
# Definition 
def original_function(*args, **kwargs):
    pass

# Decoration
original_function = decorator_name(original_function) 
```

or in a more Pythonic notation 
```python 
# Definition + Decoration 
@decorator_name
def original_function(*args, **kwargs):
    pass
```

Note that since the decorator is, after all, just a function, you can also decorate it creating higher-order   
decorators that take for instance arguments. However, I find this to be easier to understand it in terms of   
their class-representation which is the subject of the next section.

---


## Section 2: Decorators as classes 

The data-model of Python suggests that everything is an object. Let's see...

In [ ]:
def simple_function(): 
    pass

class SimpleClass(object): 
    pass

print("Function 'dir': {}\n".format(dir(simple_function)))
print("Class    'dir': {}\n".format(dir(SimpleClass)))

### This means that I can use the representation of a class instead of function declaration. In fact, a function is nothing but a class that implements the `__call__` method

In [ ]:
def simple_function(x, y):
    return x + y 

class SimpleClass(object):
    def __call__(self, x, y):
        return x + y

# No difference in their action 
SimpleClass()(1,2) == simple_function(1,2)

### That said, I will transform the decorator of the previous section in this form 

In [ ]:
class TheSimplestDecorator(object):
    def __init__(self, callback):
        print("INFO: Decorator-class has been initialized")
        self.callback = callback
        
        # This line does the 'magic' as 'functools.wraps' decorator did before
        functools.update_wrapper(self, callback)
    
    def __call__(self, *args, **kwargs):
        print("- [Decorator-class]: Arguments parsed : {}".format(args))
        print("- [Decorator-class]: Keyword arguments: {}".format(kwargs))
        return self.callback(*args, **kwargs)
    
@TheSimplestDecorator
def add_two_numbers(x, y):
    """Addition of two numbers"""
    return x + y    

### Note that the decoration happens at definition time!! Not in runtime-execution (callback)!!

In [ ]:
print("Decorated (class-implementation) function: ")
print("- docstring    : %s"%add_two_numbers.__doc__)
print("- name         : %s\n"%add_two_numbers.__name__)
print("Our result 1 + 2 = {}".format(add_two_numbers(1,2))) 

### Remarks

Using a class-pattern to create a decorator might seem a bit too much. Indeed for many simple cases this is definitely not necessary. However,  
the 'class' allows me to 'store' and retrieve additional information regarding the 'status' of the call!

In [ ]:
class LessSimpleDecorator(object):
    """Same decorator as before with the difference that now I keep track 
    of the numbers that the 'decorated' function has been called!!"""
    def __init__(self, callback):
        print("INFO: Decorator-class has been initialized")
        self.callback = callback
        
        # This line does the 'magic' as 'functools.wraps' decorator did before
        functools.update_wrapper(self, callback)
        
        # Counter of the function call 
        self.times_called = 0
    
    def __call__(self, *args, **kwargs):
        self.times_called += 1
        print("- [Decorator-class]: Arguments parsed : {}".format(args))
        print("- [Decorator-class]: Keyword arguments: {}".format(kwargs))
        return self.callback(*args, **kwargs)
    
@LessSimpleDecorator
def add_two_numbers(x, y):
    """Addition of two numbers"""
    return x + y    


for i in range(10):
    add_two_numbers(1,1)
print("\nNumber of times that the function has been called: %s"%add_two_numbers.times_called)

### So far I have discussed the decoration of plain functions. How about decorating a method of a class instead??

In [ ]:
class SimpleClass(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    @LessSimpleDecorator
    def addition(self):
        return self.x + self.y 
    
SC = SimpleClass(1,2)

### Everything seems ok... but actually is not. 

The Python interpreter checks only a handful of things when translating your code into a machine language. Actually it does  
not care about types or parsing objects, and just creates the bare-bone skeleton of you implementation. All type-checking   
and corresponding restrictions are left at runtime-execution. This very last fact, is one of the reasons why Python is slow:  
in statically-type languages, many things get's checked already at compile time. Asserting that your implementation fulfills   
the restrictions at compilation-time, removes the need of cross-validation (imagine multiple if statements, switch commands and   
bound's checking) during runtime. This is something that you can also do in Python, by working directly to the Cython layer,   
but I will not discuss it here.  


In [ ]:
try:
    SC.addition()
except TypeError as err:
    # Uncomment if you want to see the full error-message
    print("\n\n--> TypeError raised!!")

### Mmm... this had to be expected:

`addition` is a method of the `SimpleClass`, thus its first argument is actually the corresponding instance!! We can overpass this problem   
by invoking the so-called `descriptors` of the class. This is VERY technical, goes quite deep and deserves its own tutorial... 

In [ ]:
class NotThatSimpleDecorator(object):
    """Same decorator as before with the difference that now I keep track 
    of the numbers that the 'decorated' function has been called!!"""
    def __init__(self, callback):
        print("INFO: Decorator-class has been initialized")
        self.callback = callback
        
        # This line does the 'magic' as 'functools.wraps' decorator did before
        functools.update_wrapper(self, callback)
        
        # Counter of the function call 
        self.times_called = 0
    
    def __get__(self, instance, instancetype):
        """With this overload we allow the decorator to act on methods of other classes"""
        print("- Info: Instancetype = %s"%instancetype)
        print("- Info: Instance     = %s"%instance) 
        return functools.partial(self.__call__, instance)
    
    def __call__(self, *args, **kwargs):
        self.times_called += 1
        print("- [Decorator-class]: Arguments parsed : {}".format(args))
        print("- [Decorator-class]: Keyword arguments: {}".format(kwargs))
        return self.callback(*args, **kwargs)
    
class SimpleClass(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    @NotThatSimpleDecorator
    def addition(self):
        """An addition of the attributes"""
        return self.x + self.y 
    
    @NotThatSimpleDecorator
    def addition_with_arguments(self, *args, **kwargs):
        return self.x + self.y + sum(args) + sum(kwargs.values())
    
    @staticmethod
    @NotThatSimpleDecorator
    def addition_static(x,y):
        """A static example (no instance is needed!!)"""
        return x + y 
    
    def __str__(self):
        return "SimpleClass holding x = {0} and y = {1}".format(self.x, self.y) 
    
    def __repr__(self):
        return self.__str__()

In [ ]:
# Create an instance 
example = SimpleClass(1,2); 
print(example)

# And now we can call without problems 
print("Method")
print("Method of class result: %s\n"%example.addition())
print("Staticmethod")
print("Staticmethod of class result: %s"%example.addition_static(1,2))

### To explain a bit what is happening 

- Method case:
    - First there is a call of the `__get__` method
    - This freezed the function to a view of the instance 
    - Subsequently the `__call__` method of the decorator is called as usual
- Staticmethod case
    - In this case the decorator acts as in the normal function-decoration

In [ ]:
# The __get__ method is called whenever you try to access the method 'addition' 
example.addition

### As you can see, the `__get__` returns  a partially evaluated function. It is this (partial) function that we eventually call (see the arguments of the output!)

In [ ]:
example.addition()  # As long as you put the parenthesis the __call__ method is employed!!

### For completenss, I close this part discussing again the 'losing' information concept

In [ ]:
# Regarding the staticmethod no problems. Everything is properly resolved
print("Decorated (class-implementation) static method: ")
print("- docstring    : %s"%example.addition_static.__doc__)
print("- name         : %s\n"%example.addition_static.__name__)

# However for methods of the class things are very different (ERROR RAISED)
print("Decorated (class-implementation) class-method: ")
print("- docstring    : %s"%example.addition.__doc__)      
print("- name         : %s\n"%example.addition.__name__)  # ERROR AT THIS POINT 

### Although you have a raised exception, keep in mind that the information that you look for is never lost. It's just hidden!

In [ ]:
hold_the_result_of_get = example.addition
print("\nDecorated (class-implementation) class-method: ")
print("- docstring    : %s"%hold_the_result_of_get.func.__self__.__doc__)   
print("- name         : %s\n"%hold_the_result_of_get.func.__self__.__name__)

Note: There are much better ways to adjust this information, but I don't want to break your balls even more.

### Of course calling a method that receives arguments is not a problem

In [ ]:
example.addition_with_arguments(1,2,3, x=10, z=100)

## - Decorators with Arguments 

It is quite common that we need to have a decorator that takes arguments. In continuation of our example,  
let's assume that the want to apply the decorator and at the same time manipulate its printout message.  
Since we have introduced the class-notation of the decorator this becomes rather easy to do, having in   
mind that the initialization of the decorator now should not invoke the callback function, but rather   
the arguments that we want to pass. With few modifications, our decorator looks like:

In [ ]:
class DecoratorWithArguments(object):
    """Same decorator as before with the difference that now I keep track 
    of the numbers that the 'decorated' function has been called!!"""
    def __init__(self, printout_prefix):
        self.printout = printout_prefix
  
    def __call__(self, callback):
        """Note that the argument now is the target function and that you 
        return a 'wrapper' and not the result!!"""
        @functools.wraps(callback)
        def _wrapper(*args, **kwargs):
            print("- [{0}]: Arguments passed: {1}".format(self.printout, args))
            print("- [{0}]: Keywords passed: {1}".format(self.printout, kwargs))
            return callback(*args, **kwargs)
        return _wrapper

    
class SimpleClass(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    @DecoratorWithArguments(printout_prefix="title-1")
    def addition(self):
        """An addition of the attributes"""
        return self.x + self.y 
    
    @DecoratorWithArguments(printout_prefix="title-2")
    def addition_with_arguments(self, *args, **kwargs):
        return self.x + self.y + sum(args) + sum(v for v in kwargs.values())
    
    @staticmethod
    @DecoratorWithArguments(printout_prefix="title-3")
    def addition_static(x,y):
        """A static example (no instance is needed!!)"""
        return x + y 
    
    def __str__(self):
        return "SimpleClass holding x = {0} and y = {1}".format(self.x, self.y) 
    
    def __repr__(self):
        return self.__str__()
    
    
example = SimpleClass(1,2)
print("Result simple addition: %s"%example.addition())
print("Result static addition: %s"%example.addition_static(1,2))
print("Result static addition with arguments: %s"%example.addition_with_arguments(1,2, r=3))

#### Note that the `__get__` descriptor is not needed in that case: after all we explicitly return the 'wrapped' function directly
---

## Section 3: Creation of interfaces

Having introduced and explained to some extend decorators, my last section will be devoted on implementation patterns   
that you might find useful for your codebase. By no means, my strategy should be considered as the correct one. In fact  
there is no `correct` way of designing your architecture. 

My projects are based on 'modular' code structure. This means that I usually develop applications that can be used either    
as standalone applications or can be inserted within a greater multi-purpose scheme. To do so, I found myself going into   
the direction of interfaces. In simple words, an interface is a way to allow specification (subclassing) of a class  enforcing   
some principles, or an API. No matter what is the corresponding implementation, for instance, I can be ensured that **any**   
class that derives from a `BaseClass` will implement a specific method `export_to_file`. Then, decorators can be   
used in order to expand a bit their usage or ensure the existence of other requirements, such as libraries, python-version, e.t.c..   

The combination of decorators and interfaces allows you to minimize your code and force you to create concepts, which in my opinion   
is the most important thing towards a proper development of an application. 




### Decorator's abstraction 

In the previous discussion, I have worked with a handful of decorators (that just printout something). Imagine   
that you create 10 or 20 of them. There will be a lot of dublicate code which we can take care of as follows. 

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class BaseDecoratorWithoutArguments(ABC):
    """Abstract class of any decorator that takes no arguments. """
    def __init__(self, callback):
        self.callback = callback
        
        # This line does the 'magic' as 'functools.wraps' decorator
        functools.update_wrapper(self, callback)
        
    def __get__(self, instance, instancetype):
        """With this overload we allow the decorator to act on methods of other classes"""
        return functools.partial(self.__call__, instance)
    
    # When you put the @abstractmethod decorator in a function you enforce that 
    # all derived classes must implement this function in their body
    @abstractmethod
    def __call__(self, *args, **kwargs):
        """All decorators are enforced to implement this function"""
        pass

        
class BaseDecoratorWithArguments(ABC):
    """Abstract class of any decorator that takes arguments"""    
    def __init__(self, **decorator_arguments):
        self.__dict__.update(decorator_arguments)
        
    @abstractmethod
    def __call__(self, callback):
        """All decorators are enforced to implement this function"""
        pass 

### Now all my decorators will be subclasses of those abstractions. Let's create one for each case

In [ ]:
class ElapsedTimeDecorator(BaseDecoratorWithoutArguments):
    """Decorator to measure 1 time the execution of a callback"""
    def __call__(self, *args, **kwargs):        
        timestamp = datetime.now()
        result = self.callback(*args, **kwargs)
        print("- Elapsed time for running '{0}': {1}".format(
            self.callback.__name__, datetime.now()-timestamp))
        return result
    
class ElapsedTimeDecoratorMultiple(BaseDecoratorWithArguments):
    """
    Decorator to measure N times the execution of a callback
    Initialization using 'timing_rounds' (instead of rewritting the __init__)
    """    
    def __call__(self, callback):
        @functools.wraps(callback)
        def _wrapper(*args, **kwargs):
            for loop in range(self.timing_rounds):
                timestamp = datetime.now()
                result = callback(*args, **kwargs)
                print("- Elapsed time for running '{0}': {1}".format(
                    callback.__name__, datetime.now()-timestamp))
            return result
        return _wrapper
    
class TotalFunctionCalls(BaseDecoratorWithoutArguments):
    """Decorator to count how many times a callback is called. Here 
    we need to explicitly write the __init__ so that we can 
    introduce the 'counter'"""
    def __init__(self, callback):
        super(TotalFunctionCalls, self).__init__(callback)
        self.count_calls = 0
    
    def __call__(self, *args, **kwargs):
        self.count_calls += 1
        return self.callback(*args, **kwargs) 

### Some trivial examples

In [ ]:
print(80*"=")
@ElapsedTimeDecorator
def test1(x,y):
    return x+y
print("Result: %s"%test1(1,2))
print(80*"=")
@ElapsedTimeDecoratorMultiple(timing_rounds=5)
def test2(x,y, *args, **kwargs):
    return x+y + sum(args) + sum(kwargs.values())
print("Result: %s"%test2(1,2,1,2,3, r=3))
print(80*"=")
@TotalFunctionCalls
def function():
    return None
for i in range(5): function()
print("Total calls of 'function': %s"%function.count_calls)

### Lastly, since the decorator acts at definition time, have in mind that you can dynamically employ it by so-called 'monkey-patching'

In [ ]:
# A simple undecorated class 
class Person(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
        
    def add(self, *args, **kwargs):
        return self.x + self.y + sum(args) + sum(kwargs.values())
    
P = Person(1,2)
print("[Undecorated] Result: %s"%P.add(10, 20, value=50))

# Monkey-patching 
P.add = ElapsedTimeDecoratorMultiple(timing_rounds=5)(P.add) # Note the parenthesis!!
print("[Decorated] Result: %s"%P.add(10, 20, value=50))

----

## Section 4: Implementation of a cache (+questions)

In [ ]:
class CacheDecorator(BaseDecoratorWithoutArguments):
    """A simple implementation of a cache"""
    def __init__(self, callback):
        super(CacheDecorator, self).__init__(callback)
        self._cache = {}
        
    def __call__(self, *args, **kwargs):
        """Implementation of the cache pattern"""
        assert not kwargs, "Dictionary is not hashable"

        try:
            # Check if the '_cache' contains the requested evaluation
            result = self._cache[tuple(args)]  
            print("- Result retrieved from the cache")
        except KeyError as error:
            # Explicitly call the function and store the result 
            result = self._cache[tuple(args)] = self.callback(*args, **kwargs)
            print("- Result evaluated explicitly")
        except Exception as error:
            raise Exception(error) 
        return result 

#### A simple yet powerful example of the cache benefits 

- No cache involved

In [ ]:
def computationally_expensive_function(x, y):
    """Routine to represent an costly function"""
    time.sleep(0.2) # In seconds 
    return x * y

print("No cache implementation")
timestamp = datetime.now() 
computationally_expensive_function(2,1)
computationally_expensive_function(2,1)
computationally_expensive_function(1,2)
computationally_expensive_function(1,2)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
print("--> Total elapsed time: {}\n".format(datetime.now()-timestamp))


- Cache decorator employed

In [ ]:
@CacheDecorator
def computationally_expensive_function(x, y):
    """Routine to represent an costly function"""
    time.sleep(0.2) # In seconds 
    return x * y

print("Cache implementation")
timestamp = datetime.now() 
computationally_expensive_function(2,1)
computationally_expensive_function(2,1)
computationally_expensive_function(1,2)
computationally_expensive_function(1,2)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
computationally_expensive_function(1,1)
print("--> Total elapsed time: {}\n".format(datetime.now()-timestamp))

### Remarks and Questions

The cache implementation is of great usage whenever you have routines that are usually called recursively and the   
arguments of the function repeat over time. Essentially you trade computational time with computational resources, since   
you effectivelly store your result in an internal structure (lookup table). These kind of trade-offs between complexity and memory are of   
significant importance and should be carefully considered whenever you design an algorithm. Note that in case of limited resources you    
can further implement clearing-cache-strategies to reduce the memory-footprint at runtime. 

In order to practice on this particular example, try to answer the following without executing the code:
```python
@CacheDecorator
def computationally_expensive_function(x, y):
    """Routine to represent an costly function"""
    time.sleep(0.2) # In seconds 
    return x * y

# Q-1: What will be the output
computationally_expensive_function(0,0)

# Q-2: What will be the output
computationally_expensive_function([1,2],2)

# Q-3: What will be the output
computationally_expensive_function(*[1,2],2)

# Q-4: What will be the output
computationally_expensive_function(1, y=2)

# Q-5: What will be the output
computationally_expensive_function(x=1, y=2)
```

**Task**: Extend the `CacheDecorator` such that you can resolve -- when necessary and possible -- the raised issues! 

**Note 1**: I would suggest that if you work on these questions, formulate a concrete answer and post it in the 
issues, contact me or send me your solution to review it. 

**Note 2**: In case you find mistakes in the notebook raise an issue in github so that we can fix it

**Note 3**: In case that you found that the notebook is too long to follow, try to estimate at which point   
you would have prefered to end and be splitted. This is important so that we can set some guidelines for future tutorials. 

**Note 4**: Create a list of topics that you would like to hear or talk about. This way we make assignments so   
that everyone prepares things that others find useful and interesting.

---